In [1]:
from datetime import datetime

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

from logger import Logger
from solver import Solver
from crossngover import CrossN


torch.manual_seed(0)

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=216,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=216,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
net = torchvision.models.mobilenet_v2(pretrained=False)
num_ftrs = net.last_channel

classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

net.classifier = nn.Linear(num_ftrs, len(classes))
net.cuda()
print()

In [3]:
# net = torchvision.models.resnet101(pretrained=True)
# net.fc

Linear(in_features=2048, out_features=1000, bias=True)

In [3]:
# num_ftrs = 1000

# classes = ('plane', 'car', 'bird', 'cat',
#         'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

# net.classifier = nn.Linear(num_ftrs, len(classes))
# net.cuda()
# print()

170500096it [01:20, 11660257.01it/s]                               

mobilenet from F:
- Evo + Cross  - processing
- Evo Only - 
- Standard - 

resnet from F:
- Evo + Cross - 
- Evo Only - 
- Standard -  

mobilenet pretrained T:
- Evo + Cross  - 
- Evo Only   - 
- Standard - 

resnet pretrained T:
- Evo + Cross  - 
- Evo Only - 
- Standard - 

In [10]:
evo_step = 10
mode = 'evo_cross'

#mode = 'evo_only'

params = {'net_name':'mobile_net',
         'preptrained':'F',
         'mode':mode,
         'evo_step':str(evo_step)}

In [11]:
experiment_note = ''
path = ''
for key in params:
    experiment_note += key +'_'+ params[key]+'\n'
    path +=  '_'+ params[key]


logger = Logger(path, experiment_note)

In [12]:
logger.path

'./experiments/2019-12-23/_mobile_net_F_Evo + Cross_10_11:46'

In [13]:
lr = 0.001

optimizer = torch.optim.SGD(net.parameters(), lr=lr, momentum=0.9)
criterion = nn.CrossEntropyLoss()
evo_optim = CrossN()

def validation(net, dataloader):
    correct = 0
    total = 0
    with torch.no_grad():
        for data in dataloader:
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return float(100 * correct / total)


solver = Solver(
    net,
    optimizer,
    logger,
    criterion,
    validation,
    evo_optim, 
    trainloader,
    testloader,
    testloader,  
    epochs=100,
    evo_step=evo_step,
    child_count=20,
    best_child_count=3,
    mode=mode,
    debug=False,
    lr=lr)

In [10]:
logger.close()

In [9]:
logger.add_post_result(f'start: {datetime.now()}')
solver.start()
logger.add_post_result(f'finish: {datetime.now()}')
logger.close()

Start training
Epoch: 0


100%|██████████| 19/19 [00:30<00:00,  1.60s/it]

best child - 10.0


Epoch: 1
[2] loss: 2.005 validation score: 25.86 %
Epoch: 2
[3] loss: 1.800 validation score: 35.75 %
Epoch: 3
[4] loss: 1.668 validation score: 39.61 %
Epoch: 4
[5] loss: 1.588 validation score: 42.74 %
Epoch: 5
[6] loss: 1.450 validation score: 44.55 %
Epoch: 6


KeyboardInterrupt: 

In [25]:
logger.close()